In [1]:
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import nltk
from nltk.corpus import stopwords
import re
import numpy as np
from bs4 import BeautifulSoup
from sklearn.svm import LinearSVC
import csv

In [2]:
data=pd.read_csv('IMDB-Dataset.csv')
dat=data.copy()
final=dat.dropna()
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/pavan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
x=final.drop(['sentiment'],axis=1)
y=final["sentiment"]
for i in y.index:
  if y[i]=="positive":
    y[i]=1
  elif y[i]=="negative":
    y[i]=0

In [4]:
review = dat['review'].loc[1]
soup = BeautifulSoup(review, "html.parser")
review = soup.get_text()

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
dataset_train, dataset_test, train_data_label, test_data_label = train_test_split(dat['review'], dat['sentiment'], test_size=0.25, random_state=42)

In [7]:
train_data_label = (train_data_label.replace({'positive': 1, 'negative': 0})).values
test_data_label  = (test_data_label.replace({'positive': 1, 'negative': 0})).values

In [8]:
corpus_train = []
corpus_test  = []
with open('finalReview.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["review", "sentiment"])
    for i in range(dataset_train.shape[0]):
        soup = BeautifulSoup(dataset_train.iloc[i], "html.parser")
        review = soup.get_text()
        review = re.sub('\[[^]]*\]', ' ', review)
        review = re.sub('[^a-zA-Z]', ' ', review)
        review = review.lower()
        review = review.split()
        review = [word for word in review if not word in set(stopwords.words('english'))]
        lem = WordNetLemmatizer()
        review = [lem.lemmatize(word) for word in review]
        review = ' '.join(review)
        writer.writerow([review, train_data_label[i]])
        corpus_train.append(review)
      
    for j in range(dataset_test.shape[0]):
        soup = BeautifulSoup(dataset_test.iloc[j], "html.parser")
        review = soup.get_text()
        review = re.sub('\[[^]]*\]', ' ', review)
        review = re.sub('[^a-zA-Z]', ' ', review)
        review = review.lower()
        review = review.split()
        review = [word for word in review if not word in set(stopwords.words('english'))]
        lem = WordNetLemmatizer()
        review = [lem.lemmatize(word) for word in review]
        review = ' '.join(review)
        writer.writerow([review, test_data_label[j]])
        corpus_test.append(review)

In [9]:
tfidf_vec = TfidfVectorizer(ngram_range=(1, 3))

In [10]:
tfidf_vec_train = tfidf_vec.fit_transform(corpus_train)
tfidf_vec_test = tfidf_vec.transform(corpus_test)

In [11]:
linear_svc = LinearSVC(C=0.5, random_state=42)
linear_svc.fit(tfidf_vec_train, train_data_label)

LinearSVC(C=0.5, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
          verbose=0)

In [12]:
predict = linear_svc.predict(tfidf_vec_test)

In [13]:
check =pd.read_csv('moviereview.csv')

In [17]:
Test_Data = check['reviews']

In [18]:
Checker_Data = tfidf_vec.transform(Test_Data)
predict_test = linear_svc.predict(Checker_Data)

In [19]:
filename = 'final_model.sav'
joblib.dump(linear_svc,filename)

['final_model.sav']

In [23]:
predict_test = pd.DataFrame(predict_test,  columns=['Sentiment'])

In [24]:
new_dataframe = pd.concat([check, predict_test], axis=1)

In [25]:
new_dataframe.to_csv('Result1.csv')

In [50]:
df = pd.read_csv('Result1.csv')

In [51]:
df

,Unnamed: 0,Movie_name,Genre,reviews,Sentiment
0,0,Peninsula,Thriller,Very good movie for youths and teenagers. That...,1
1,1,Peninsula,Thriller,Very good direction and very good acting too. ...,1
2,2,Peninsula,Thriller,"not as sequel, good for individual watch",1
3,3,Peninsula,Thriller,very good movie to watch,1
4,4,Come Play,Mystery,"I did not like Babadook, but this one is actua...",1
...,...,...,...,...,...
410,410,Bhoot,Horror,Best action of Vicky kausal,1
411,411,Bhoot,Horror,Movies has good storyline...it is an one time ...,1
412,412,Bhoot,Horror,Great horror story. Different from other horro...,1
413,413,Bhoot,Horror,Different from normal horror stories of bollyw...,1


In [53]:
Movie_name = new_dataframe.Movie_name.unique()

In [54]:
Movie_name

array(['Peninsula', 'Come Play', 'The Secret: Dare To Dream',
       'The War with grandpa', 'Parasite', '1917', 'The Outpost',
       'Little Woman', 'Birds of  prey', 'Jojo Rabbit', 'The gentleman',
       'A Beautiful Day in the neighbourhood', 'trolls world tour',
       'The Witches', "Never Steal A man's second chance", 'Greenland',
       'Antebellum', 'A secret : Dare to dream', 'The Silencing', 'Relic',
       'BloodShot', 'Sonic the Hedgehog', 'Bad Boys for life',
       'The Lasr Full measure', 'Just Mercy', 'Dolittle', 'Bombshell',
       'The Lost city', 'Life is  Beautiful', 'Good will Hunting',
       'The Shawshank redemption', 'Angry men', 'angry men',
       'The Dark Knight', 'The Matrix', 'Forest Gump', 'America Beauty',
       'The Godfather', 'the Departed', 'The Prestige', 'Gladiator',
       'Inglourious Basterds', 'Million Dollar baby', 'The Aviator',
       'Avatar', 'Big Fish', 'Meet Joe Black', 'The Town',
       "Schndler's List", 'Titanic', 'spy game', 'ps

In [55]:
Movie_name = Movie_name[:250]

In [56]:
Movie_name

array(['Peninsula', 'Come Play', 'The Secret: Dare To Dream',
       'The War with grandpa', 'Parasite', '1917', 'The Outpost',
       'Little Woman', 'Birds of  prey', 'Jojo Rabbit', 'The gentleman',
       'A Beautiful Day in the neighbourhood', 'trolls world tour',
       'The Witches', "Never Steal A man's second chance", 'Greenland',
       'Antebellum', 'A secret : Dare to dream', 'The Silencing', 'Relic',
       'BloodShot', 'Sonic the Hedgehog', 'Bad Boys for life',
       'The Lasr Full measure', 'Just Mercy', 'Dolittle', 'Bombshell',
       'The Lost city', 'Life is  Beautiful', 'Good will Hunting',
       'The Shawshank redemption', 'Angry men', 'angry men',
       'The Dark Knight', 'The Matrix', 'Forest Gump', 'America Beauty',
       'The Godfather', 'the Departed', 'The Prestige', 'Gladiator',
       'Inglourious Basterds', 'Million Dollar baby', 'The Aviator',
       'Avatar', 'Big Fish', 'Meet Joe Black', 'The Town',
       "Schndler's List", 'Titanic', 'spy game', 'ps

In [65]:
count = 1
index = 0
overall = []
while count<=250:
    temp_count = 0
    reviews = 0
    while temp_count < 3:
        reviews += new_dataframe.Sentiment[index]
        index += 1
        temp_count += 1
    if reviews/4 < 0.5:
        overall.append("Negative")
    elif reviews/4 >= 0.5:
        overall.append("positive")
    count += 1

KeyError: 415